In [1]:
import os
import sys
os.environ['SPARK_HOME']='/usr/lib/spark'
os.environ['PYLIB']=os.environ['SPARK_HOME']+'/python/lib'
sys.path.insert(0,os.environ['PYLIB']+'/py4j-0.10.7-src.zip')
sys.path.insert(1,os.environ['PYLIB']+'/pyspark.zip')

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [3]:
spark = SparkSession.builder.appName('SparkRestOps') \
.config('hive.metastore.uris', 'thrift://localhost:9083') \
.enableHiveSupport().getOrCreate()

In [4]:
# Having set the driver and driver options we should have spark representing spark session 
# available straight away
spark.version

'2.4.1'

In [5]:
sc = spark.sparkContext

In [6]:
import requests
players = requests.get('http://localhost:5000/players').json()

In [7]:
players

[{'ln': 'Tendulkar',
  'fn': 'Sachin',
  'stats': {'odi': {'matches': 463,
    'innings': 452,
    'notout': 41,
    'runs': 18426,
    'hs': 200,
    'avg': 44.83,
    'centuries': 49,
    'doublehoundreds': 1,
    'fifties': 96,
    'fours': 2016,
    'sixes': 195},
   'test': {'matches': 200,
    'innings': 329,
    'notout': 33,
    'runs': 15921,
    'hs': 248,
    'avg': 53.79,
    'centuries': 51,
    'doublehoundreds': 6,
    'fifties': 68,
    'fours': 2058,
    'sixes': 69},
   'fansaus': ['Bill Lawry', 'Ian Chappell', 'Mark Taylor'],
   'faneng': ['Tony Greig', 'Geoofrey Boycott', 'Nasser Hssain'],
   'fanwb': ['Nichael Holding', 'Tony Cozier'],
   'fanind': ['Sunil Gavaskar', 'Ravi Shastri', 'Sanja Manjarekar']}},
 {'ln': 'Kohli',
  'fn': 'Virat',
  'stats': {'odi': {'matches': 227,
    'innings': 219,
    'notout': 37,
    'runs': 10843,
    'hs': 183,
    'avg': 59.58,
    'centuries': 41,
    'doublehoundreds': 0,
    'fifties': 49,
    'fours': 1019,
    'sixes': 117},


In [8]:
players_df = spark.read.json(sc.parallelize(players))

In [9]:
players_df.show(truncate=False)

+------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|fn    |ln       |stats                                                                                                                                                                                                                                                                                                   |
+------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Sachin|Tendulkar|[[Tony Greig, Geoofrey Boycott, Na

In [10]:
players_df.printSchema()

root
 |-- fn: string (nullable = true)
 |-- ln: string (nullable = true)
 |-- stats: struct (nullable = true)
 |    |-- faneng: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- fanind: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- fansaus: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- fanwb: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- odi: struct (nullable = true)
 |    |    |-- avg: double (nullable = true)
 |    |    |-- centuries: long (nullable = true)
 |    |    |-- doublehoundreds: long (nullable = true)
 |    |    |-- fifties: long (nullable = true)
 |    |    |-- fours: long (nullable = true)
 |    |    |-- hs: long (nullable = true)
 |    |    |-- innings: long (nullable = true)
 |    |    |-- matches: long (nullable = true)
 |    |    |-- notout: long (nullable = true)
 |    |    |-- runs: long (nullable = true)
 | 

In [12]:
players_df.select('fn', 'ln', 'stats.odi.matches', 'stats.odi.centuries', 
                 'stats.test.matches', 'stats.test.centuries').show()

+------+---------+-------+---------+-------+---------+
|    fn|       ln|matches|centuries|matches|centuries|
+------+---------+-------+---------+-------+---------+
|Sachin|Tendulkar|    463|       49|    200|       51|
| Virat|    Kohli|    227|       41|     77|       25|
+------+---------+-------+---------+-------+---------+

